In [1]:
import netket as nk
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry import FermionicOperator
import numpy as np

from system_dicts import *

#systemData = b631g_H2[4]
systemData = sto3g_H2_eq

#systemData={'driver_string': 'H 0.0 0.0 0.0; H 0.0 0.0 0.734', 'basis': 'sto3g'}; n_electrons=2
#systemData={'driver_string': 'H 0.0 0.0 0.0; H 0.0 0.0 0.734', 'basis': '631g'}; n_electrons=2
#systemData={'driver_string': 'Li 0.0 0.0 0.0; H 0.0 0.0 1.54', 'basis': 'sto3g'}; n_electrons=4
#systemData={'driver_string': 'C 0.0 0.0 0.0; C 0.0 0.0 1.26', 'basis': 'sto3g'}; n_electrons=12

print(systemData)
driver = PySCFDriver(   atom=systemData["atomstring"],
                        basis=systemData["basis"])
                        
mol = driver.run()
OB = mol.one_body_integrals
TB = mol.two_body_integrals

nre = mol.nuclear_repulsion_energy

FerOp = FermionicOperator(OB, TB)
#FerOp.fermion_mode_elimination([0])
mapping = FerOp.mapping('jordan_wigner')
weights = [w[0] for w in mapping.paulis]
operators = [w[1].to_label() for w in mapping.paulis]
ha = nk.operator.PauliStrings(operators, weights)

g = nk.graph.Hypercube(n_dim=1, length=ha.hilbert.size, pbc=False)
hi = nk.hilbert.Qubit(graph=g)
assert(hi.size==ha.hilbert.size)


/Users/yapple/.pyenv/versions/miniconda3-4.7.12/envs/nkcenv/lib/python3.7/site-packages/qiskit/chemistry/__init__.py:170: DeprecationWarning: The package qiskit.chemistry is deprecated. It was moved/refactored to qiskit_nature (pip install qiskit-nature). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('chemistry', 'qiskit_nature', 'qiskit-nature')
{'molecule': 'H2', 'basis': 'sto3g', 'n_basisfuncs': 4, 'distance': 0.7348999999999997, 'atomstring': 'H 0.0 0.0 0.0; H 0.0 0.0 0.734900', 'eq': True, 'tot_energy': -1.1373060501921608, 'nuc_rep_energy': 0.720066962403048, 'n_electrons': 2}
/Users/yapple/.pyenv/versions/miniconda3-4.7.12/envs/nkcenv/lib/python3.7/site-packages/qiskit/chemistry/fermionic_operator.py:386: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob

In [2]:
res = nk.exact.lanczos_ed(ha, first_n=1, compute_eigenvectors=False)
print("Exact ground state energy = {}".format(res.eigenvalues[0] + nre))

Exact ground state energy = -1.1511621890123025


In [19]:
ha.get_conn([1,0,1,0])

(array([[1., 0., 1., 0.],
        [0., 1., 0., 1.]]),
 array([-0.24507193+0.j,  0.18092562+0.j]))

In [3]:
print(ha.to_sparse())

  (1, 1)	(-1.2223002165506713+0j)
  (1, 4)	(-0.16259078402216834+0j)
  (2, 2)	(-0.5571981480916433+0j)
  (2, 8)	(0.20616810035124639+0j)
  (3, 3)	(-1.4284427619214306+0j)
  (3, 6)	(0.09384404941711434+0j)
  (3, 9)	(0.1431033514393945+0j)
  (3, 12)	(-0.049240758498173165+0j)
  (4, 1)	(-0.16259078402216834+0j)
  (4, 4)	(-0.15654139913780638+0j)
  (5, 5)	(-0.9581961095717589+0j)
  (5, 10)	(-0.06264789271383672+0j)
  (6, 3)	(0.09384404941711434+0j)
  (6, 6)	(-0.36511674666417715+0j)
  (6, 9)	(-0.013407134215663568+0j)
  (6, 12)	(-0.11119173362333379+0j)
  (7, 7)	(-0.8157158543772463+0j)
  (7, 13)	(-0.048126984711481954+0j)
  (8, 2)	(0.20616810035124639+0j)
  (8, 8)	(0.1951525503234273+0j)
  (9, 3)	(0.1431033514393945+0j)
  (9, 6)	(-0.013407134215663568+0j)
  (9, 9)	(-0.5120051646484123+0j)
  (9, 12)	(-0.08920822110167316+0j)
  (10, 5)	(-0.06264789271383672+0j)
  :	:
  (246, 243)	(-0.4388414947363243+0j)
  (246, 246)	(3.89546937757655+0j)
  (246, 249)	(-0.013407134215663568+0j)
  (246, 252)

In [2]:
np.random.seed(seed=123)
nk.random.seed(seed=123)

ma = nk.machine.RbmSpin(hi, alpha=2)
ma.init_random_parameters(seed=1234, sigma=0.05)

op = nk.optimizer.Sgd(0.1)

chain_length = 4
sa = nk.sampler.MetropolisExchange(machine=ma, n_chains=chain_length)

n_up = []
tries = 20000
for i in range(tries):
    n_up = []
    sa = nk.sampler.MetropolisExchange(machine=ma, n_chains=chain_length)
    for ss in sa.samples(1):
        for s in ss:
             #print(s, list(s).count(1))
             n_up.append(int(list(s).count(1)))
    if n_up.count(systemData["n_electrons"]) == chain_length: print('found after %d tries' %(i)); break
if i+1==tries:
    print('not found')
else:
    for s in sa.samples(2):
        print(s)

vmc = nk.variational.Vmc(
    hamiltonian=ha,
    sampler=sa,
    optimizer=op,
    n_samples=100000,
    diag_shift=0.1,
    use_iterative=True,
    method='Sr')

found after 4 tries
[[1. 1. 0. 0.]
 [1. 0. 0. 1.]
 [1. 1. 0. 0.]
 [0. 1. 1. 0.]]
[[1. 0. 0. 1.]
 [1. 1. 0. 0.]
 [0. 1. 1. 0.]
 [1. 0. 0. 1.]]


In [3]:
ma.n_par

44

In [3]:
I=[]
E=[]
for it in vmc.iter(1000,5):
    print(it,vmc.energy, vmc.energy.mean + nre)
    I.append(it)
    E.append(vmc.energy.mean.real)

0 (-1.00801 - 0.00001i) ± 0.00090 [var=0.24077, R=0.99999] (-0.2879471638411294-1.431822968917726e-05j)


KeyboardInterrupt: 